In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time

# CBAM (Convolutional Block Attention Module) Layer
class CBAM(layers.Layer):
    def __init__(self):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention()
        self.spatial_attention = SpatialAttention()
    
    def call(self, inputs):
        channel_att = self.channel_attention(inputs)
        spatial_att = self.spatial_attention(inputs)
        return tf.multiply(inputs, tf.expand_dims(tf.expand_dims(channel_att, axis=1), axis=1)) * spatial_att

# Channel Attention Layer
class ChannelAttention(layers.Layer):
    def __init__(self):
        super(ChannelAttention, self).__init__()
        self.global_avgpool = layers.GlobalAveragePooling2D()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        avg_pool = self.global_avgpool(inputs)
        dense1_out = self.dense1(avg_pool)
        attention = self.dense2(dense1_out)
        return attention

# Linear Attention Layer
class LinearAttention(layers.Layer):
    def __init__(self):
        super(LinearAttention, self).__init__()
        self.dense = layers.Dense(1)
    
    def call(self, inputs):
        return tf.nn.sigmoid(self.dense(inputs))

# NonLinear Attention Layer
class NonLinearAttention(layers.Layer):
    def __init__(self):
        super(NonLinearAttention, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        dense1_out = self.dense1(inputs)
        attention = self.dense2(dense1_out)
        return attention

def configure_model(layers_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False):
    if debugflag:
        model = tf.keras.Sequential([
            layers.Conv1D(32, kernel_size=2, activation='relu', input_shape=(2, 1)),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')
        ])
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        return model
        
    model = tf.keras.Sequential()

    for layer_config in layers_config:
        layer_type = layer_config[0]

        if layer_type == 'dense':
            neurons, activation = layer_config[1], layer_config[2]
            if len(layer_config) > 3 and layer_config[3] == 'input':
                input_shape = layer_config[4]
                model.add(layers.Dense(neurons, activation=activation, input_shape=(input_shape,)))
            else:
                model.add(layers.Dense(neurons, activation=activation))
        elif layer_type == 'dropout':
            rate = layer_config[1]
            model.add(layers.Dropout(rate))
        elif layer_type == 'activation':
            activation = layer_config[1]
            model.add(layers.Activation(activation))
        elif layer_type == 'conv2d':
            filters, kernel_size, strides, activation = layer_config[1], layer_config[2], layer_config[3], layer_config[4]
            model.add(layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, activation=activation))
        elif layer_type == 'maxpool2d':
            pool_size = layer_config[1]
            model.add(layers.MaxPool2D(pool_size=pool_size))
        elif layer_type == 'rnn':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.SimpleRNN(units, return_sequences=return_sequences))
        elif layer_type == 'lstm':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.LSTM(units, return_sequences=return_sequences))
        elif layer_type == 'gru':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.GRU(units, return_sequences=return_sequences))
        elif layer_type == 'attention':
            model.add(layers.MultiHeadAttention(num_heads=layer_config[1], key_dim=layer_config[2]))
        elif layer_type == 'cbam':
            model.add(CBAM())
        elif layer_type == 'channel_attention':
            model.add(ChannelAttention())
        elif layer_type == 'linear_attention':
            model.add(LinearAttention())
        elif layer_type == 'nonlinear_attention':
            model.add(NonLinearAttention())

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


def train_model(model, X_train, y_train, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2", total_epoch=50,ea=False,eamonitor="val_loss"):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')
    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor=eamonitor, patience=10, min_delta=0.001, verbose=1)


    start_time = time.time()
    if ea:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback,early_stopping])
    else:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2",threshold=0.5):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time
    
    metrics = calculate_metrics(predictions, y_test,threshold)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels, threshold=0.5):
    #threshold = 0.5  # Define the threshold
    interpredictions=predictions.flatten()
    correct_predictions = np.sum(np.abs(true_labels - interpredictions) <= threshold)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions  # Final accuracy
    mse = mean_squared_error(true_labels, interpredictions)  # Mean Squared Error
    mae = mean_absolute_error(true_labels, interpredictions)  # Mean Absolute Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    f1 = f1_score(true_labels.round(), interpredictions.round(), average='micro')  # F1 Score

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }


In [3]:
# Load dataset
dataset_name = "integration_with_sin_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True,eamonitor='loss')

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 702.1207 - mae: 18.8905 - val_loss: 599.7911 - val_mae: 18.0054
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 584.8125 - mae: 17.7242 - val_loss: 509.8598 - val_mae: 15.7890
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 477.1259 - mae: 15.0537 - val_loss: 433.3472 - val_mae: 13.2504
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 424.1765 - mae: 12.7123 - val_loss: 400.2396 - val_mae: 11.3590
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 397.7852 - mae: 11.0661 - val_loss: 386.4954 - val_mae: 10.4464
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 380.4858 - mae: 10.2460 - val_loss: 376.6356 - val_mae: 10.0801
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 367.7502 - mae: 9.6215 - val_loss: 365.5135 - val_mae: 9.5566
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 360.2931 - mae: 9.3799 - val_loss: 351.7985 - val_mae: 9.3206
Epoch 9/50
5

In [4]:
# Load dataset
dataset_name = "integration_with_cos_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True,eamonitor='loss')

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1120.5049 - mae: 22.6244 - val_loss: 1000.0818 - val_mae: 21.9712
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 984.0762 - mae: 21.5501 - val_loss: 872.3124 - val_mae: 20.6054
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 798.9661 - mae: 19.5492 - val_loss: 442.1930 - val_mae: 14.8461
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 343.9647 - mae: 12.8784 - val_loss: 85.6743 - val_mae: 6.1356
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 58.9921 - mae: 4.8970 - val_loss: 16.5884 - val_mae: 2.5619
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.7825 - mae: 2.3293 - val_loss: 6.4923 - val_mae: 1.5505
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 5.8499 - mae: 1.4334 - val_loss: 4.4062 - val_mae: 1.0966
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3.8055 - mae: 1.0249 - val_loss: 2.8268 - val_mae: 0.7955
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 

In [5]:
# Load dataset
dataset_name = "integration_with_tan_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True,eamonitor='loss')

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 24937218.0000 - mae: 164.4513 - val_loss: 11124270.0000 - val_mae: 211.5640
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 16267937.0000 - mae: 186.3065 - val_loss: 11124747.0000 - val_mae: 211.6667
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6713361.5000 - mae: 127.4258 - val_loss: 11125272.0000 - val_mae: 211.8690
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 30235720.0000 - mae: 204.9952 - val_loss: 11125959.0000 - val_mae: 212.2452
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 19692512.0000 - mae: 160.4022 - val_loss: 11126791.0000 - val_mae: 212.8453
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 62604656.0000 - mae: 251.9180 - val_loss: 11126766.0000 - val_mae: 212.8203
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 20702428.0000 - mae: 160.0341 - val_loss: 11127984.0000 - val_mae: 213.9438
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 

In [6]:
# Load dataset
dataset_name = "normal_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True,eamonitor='loss')

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2837 - mae: 0.5027 - val_loss: 0.2510 - val_mae: 0.5002
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2518 - mae: 0.4999 - val_loss: 0.2507 - val_mae: 0.4998
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2523 - mae: 0.5003 - val_loss: 0.2515 - val_mae: 0.4992
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2518 - mae: 0.4999 - val_loss: 0.2503 - val_mae: 0.5000
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2509 - mae: 0.4994 - val_loss: 0.2508 - val_mae: 0.4995
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2511 - mae: 0.4993 - val_loss: 0.2500 - val_mae: 0.4993
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2506 - mae: 0.4990 - val_loss: 0.2502 - val_mae: 0.5000
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2512 - mae: 0.5000 - val_loss: 0.2510 - val_mae: 0.4993
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2509 

In [7]:
# Load dataset
dataset_name = "binomial_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True,eamonitor='loss')

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3961 - mae: 0.5568 - val_loss: 0.2533 - val_mae: 0.5010
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2542 - mae: 0.5000 - val_loss: 0.2548 - val_mae: 0.4987
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2527 - mae: 0.5003 - val_loss: 0.2510 - val_mae: 0.5004
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2532 - mae: 0.4993 - val_loss: 0.2602 - val_mae: 0.4981
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2554 - mae: 0.4997 - val_loss: 0.2535 - val_mae: 0.5010
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2534 - mae: 0.4998 - val_loss: 0.2543 - val_mae: 0.5010
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2537 - mae: 0.5000 - val_loss: 0.2507 - val_mae: 0.5003
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2522 - mae: 0.5000 - val_loss: 0.2531 - val_mae: 0.4988
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2521 

In [8]:
# Load dataset
dataset_name = "gamma_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True,eamonitor='loss')

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3400 - mae: 0.5239 - val_loss: 0.2535 - val_mae: 0.4999
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2528 - mae: 0.5003 - val_loss: 0.2509 - val_mae: 0.5002
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2530 - mae: 0.5002 - val_loss: 0.2538 - val_mae: 0.4997
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2533 - mae: 0.5004 - val_loss: 0.2520 - val_mae: 0.5002
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2521 - mae: 0.5004 - val_loss: 0.2565 - val_mae: 0.5003
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2524 - mae: 0.5002 - val_loss: 0.2547 - val_mae: 0.4996
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2527 - mae: 0.4999 - val_loss: 0.2525 - val_mae: 0.5003
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2517 - mae: 0.4998 - val_loss: 0.2511 - val_mae: 0.5002
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2518 

In [9]:
# Load dataset
dataset_name = "poisson_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True,eamonitor='loss')

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2722 - mae: 0.5011 - val_loss: 0.2631 - val_mae: 0.5018
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2532 - mae: 0.4996 - val_loss: 0.2502 - val_mae: 0.4999
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2516 - mae: 0.4997 - val_loss: 0.2502 - val_mae: 0.4996
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2515 - mae: 0.4996 - val_loss: 0.2507 - val_mae: 0.5002
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2515 - mae: 0.4999 - val_loss: 0.2502 - val_mae: 0.4998
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2509 - mae: 0.4989 - val_loss: 0.2546 - val_mae: 0.5008
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2514 - mae: 0.5001 - val_loss: 0.2513 - val_mae: 0.4992
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2510 - mae: 0.4997 - val_loss: 0.2509 - val_mae: 0.5002
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2512 

In [10]:
# Load dataset
dataset_name = "exponential_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True,eamonitor='loss')

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2817 - mae: 0.5016 - val_loss: 0.2616 - val_mae: 0.4997
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2557 - mae: 0.5000 - val_loss: 0.2506 - val_mae: 0.5000
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2528 - mae: 0.5003 - val_loss: 0.2503 - val_mae: 0.5000
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2522 - mae: 0.5001 - val_loss: 0.2513 - val_mae: 0.4998
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2525 - mae: 0.4999 - val_loss: 0.2507 - val_mae: 0.5001
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2531 - mae: 0.5000 - val_loss: 0.2517 - val_mae: 0.5001
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2519 - mae: 0.5007 - val_loss: 0.2543 - val_mae: 0.4998
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2522 - mae: 0.5000 - val_loss: 0.2511 - val_mae: 0.4999
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2513 